# Bigtable
[Bigtable](https://cloud.google.com/bigtable) is a key-value and wide-column store, ideal for fast access to structured, semi-structured, or unstructured data.


## Setting up

To run this notebook, you will need a [Google Cloud Project](https://developers.google.com/workspace/guides/create-project), a [Bigtable instance](https://cloud.google.com/bigtable/docs/creating-instance), and [Google credentials](https://developers.google.com/workspace/guides/create-credentials).

In [ ]:
%pip install langchain-google-bigtable

## Querying for Documents from Bigtable
For more details on connecting to a Bigtable table, please check the [Python SDK documentation](https://cloud.google.com/python/docs/reference/bigtable/latest/client).

In [ ]:
from langchain_google_bigtable import BigtableLoader

instance_id = "my_instance"
table_id = "my_table"

## Create the Loader

In [ ]:
loader = BigtableLoader(
    instance_id,
    table_id,
)

### Load from table

You can fetch the documents by calling the `lazy_load` method that returns an Iterator of documents.

In [ ]:
for doc in loader.lazy_load():
    print(doc)
    break

page_content='address: 8301 Hollister Ave\nalias: None\ncheckin: 12PM\ncheckout: 4PM\ncity: Santa Barbara\ncountry: United States\ndescription: Located on 78 acres of oceanfront property, this resort is an upscale experience that caters to luxury travelers. There are 354 guest rooms in 19 separate villas, each in a Spanish style. Property amenities include saline infinity pools, a private beach, clay tennis courts, a 42,000 foot spa and fitness center, and nature trails through the adjoining wetland and forest. The onsite Miro restaurant provides great views of the coast with excellent food and service. With all that said, you pay for the experience, and this resort is not for the budget traveler.  In addition to quoted rates there is a $25 per day resort fee that includes a bottle of wine in your room, two bottles of water, access to fitness center and spa, and internet access.\ndirections: None\nemail: None\nfax: None\nfree_breakfast: True\nfree_internet: False\nfree_parking: False\n

## Limiting the returned rows
There are two ways to limit the returned rows:
1. Using a [filter](https://cloud.google.com/python/docs/reference/bigtable/latest/row-filters)
2. Using a [row_set](https://cloud.google.com/python/docs/reference/bigtable/latest/row-set#google.cloud.bigtable.row_set.RowSet)

In [ ]:
import google.cloud.bigtable.row_filters as row_filters

filter_loader = BigtableLoader(
    instance_id, table_id, filter=row_filters.ColumnQualifierRegexFilter(b"os_build")
)


from google.cloud.bigtable.row_set import RowSet

row_set = RowSet()
row_set.add_row_range_from_keys(
    start_key="phone#4c410523#20190501", end_key="phone#4c410523#201906201"
)

row_set_loader = BigtableLoader(
    instance_id,
    table_id,
    row_set=row_set,
)

## Custom client
The client created by default is the default client, using only admin=True option. To use a non-default, a [custom client](https://cloud.google.com/python/docs/reference/bigtable/latest/client#class-googlecloudbigtableclientclientprojectnone-credentialsnone-readonlyfalse-adminfalse-clientinfonone-clientoptionsnone-adminclientoptionsnone-channelnone) can be passed to the constructor.

In [ ]:
from google.cloud import bigtable

custom_client_loader = BigtableLoader(
    instance_id,
    table_id,
    client=bigtable.Client(...),
)

## Custom content
The BigtableLoader assumes there is a column family called `langchain`, that has a column called `content`, that contains values encoded in UTF-8. These defaults can be changed like so:

In [ ]:
from langchain_google_bigtable import Encoding

custom_content_loader = BigtableLoader(
    instance_id,
    table_id,
    content_encoding=Encoding.ASCII,
    content_column_family="my_content_family",
    content_column_name="my_content_column_name",
)

## Metadata mapping
By default, the `metadata` map on the `Document` object will contain a single key, `rowkey`, with the value of the row's rowkey value. To add more items to that map, use metadata_mapping.

In [ ]:
from langchain_google_bigtable import MetadataMapping
import json

metadata_mapping_loader = BigtableLoader(
    instance_id,
    table_id,
    metadata_mappings=[
        MetadataMapping(
            column_family="my_int_family",
            column_name="my_int_column",
            metadata_key="key_in_metadata_map",
            encoding=Encoding.INT_BIG_ENDIAN,
        ),
        MetadataMapping(
            column_family="my_custom_family",
            column_name="my_custom_column",
            metadata_key="custom_key",
            encoding=Encoding.CUSTOM,
            custom_decoding_func=lambda input: json.loads(input.decode()),
            custom_encoding_func=lambda input: str.encode(json.dumps(input)),
        ),
    ],
)

## Using the saver
It is possible to save documents into Bigtable using the BigtableSaver. The BigtableSaver constructor is very similar to the BigtableLoader's one.

In [ ]:
from langchain_google_bigtable import BigtableSaver
from langchain_core.documents import Document

saver = BigtableSaver(
    instance_id,
    table_id,
    client=bigtable.Client(...),
    content_encoding=Encoding.ASCII,
    content_column_family="my_content_family",
    content_column_name="my_content_column_name",
    metadata_mappings=[
        MetadataMapping(
            column_family="my_int_family",
            column_name="my_int_column",
            metadata_key="key_in_metadata_map",
            encoding=Encoding.INT_BIG_ENDIAN,
        ),
        MetadataMapping(
            column_family="my_custom_family",
            column_name="my_custom_column",
            metadata_key="custom_key",
            encoding=Encoding.CUSTOM,
            custom_decoding_func=lambda input: json.loads(input.decode()),
            custom_encoding_func=lambda input: str.encode(json.dumps(input)),
        ),
    ],
)

saver.add_documents([Document(), Document()])
saver.delete([Document(), Document()])